# SPARQL query

In [103]:
import pandas as pd
import rdflib
from SPARQLWrapper import SPARQLWrapper, JSON, CSV

### Input parameters

In [104]:
#latitude
#longitude
#radius

#### Example setting

In [105]:
# endpoint
endpoint = 'https://query.wikidata.org/sparql'

# query variables
latitude  = '49.487777777'
longitude = '8.466111111'
#obj_class = '' 
radius    = '15'
limit     = '30'

### Query

Description of the retrieved variables in the sparql query
- ?x : Resourcee of the thing (a specific street, chruch, etc.) we search
    - ?xLabel : Label of that resource
    - ?otherLocation : Coordinates of the respective resource
- ?somebody : identifier of the person the resource is named after
    - ?somebodyLabel : Label of that person
- ?classLabel : What kind of thing are we looking at (church, street, venue...)
- ?dist : distance between current location and the location of the retrieved resource



In [127]:

# Query for the table
query = '''
        prefix bd:       <http://www.bigdata.com/rdf#>
        prefix geo:      <http://www.opengis.net/ont/geosparql#>
        prefix wd:       <http://www.wikidata.org/entity/>
        prefix wdt:      <http://www.wikidata.org/prop/direct/>
        prefix wikibase: <http://wikiba.se/ontology#>


        SELECT ?xLabel ?x ?somebodyLabel ?somebody ?classLabel ?otherLocation ?dist
        WHERE
        {{
          BIND('Point({longitude} {latitude})'^^geo:wktLiteral AS ?currentLocation).
          SERVICE wikibase:around {{
              ?x wdt:P625 ?otherLocation. 
              bd:serviceParam wikibase:center ?currentLocation. 
              bd:serviceParam wikibase:radius '{radius}'. 
          }}
          # x is named after
          ?x wdt:P138 ?somebody .

          # which Class is x (e.g. Street, church)
          ?x wdt:P31  ?class . 
          
          # x is a Street
          #?x wdt:P31 wd:Q79007 .

          # hat Staat Deutschland
          # ?x wdt:P17  ?Q183 .
          
          # Retrieve Labels
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "de". }} 
          
          # Compute Distance
          # BIND(geof:distance(?otherLocation, ?currentLocation) as ?dist) 
        }} 

        LIMIT {limit}
        #ORDER BY ?dist
        
        '''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)
print(query)


        prefix bd:       <http://www.bigdata.com/rdf#>
        prefix geo:      <http://www.opengis.net/ont/geosparql#>
        prefix wd:       <http://www.wikidata.org/entity/>
        prefix wdt:      <http://www.wikidata.org/prop/direct/>
        prefix wikibase: <http://wikiba.se/ontology#>


        SELECT ?xLabel ?x ?somebodyLabel ?somebody ?classLabel ?otherLocation ?dist
        WHERE
        {
          BIND('Point(8.466111111 49.487777777)'^^geo:wktLiteral AS ?currentLocation).
          SERVICE wikibase:around {
              ?x wdt:P625 ?otherLocation. 
              bd:serviceParam wikibase:center ?currentLocation. 
              bd:serviceParam wikibase:radius '15'. 
          }
          # x is named after
          ?x wdt:P138 ?somebody .

          # which Class is x (e.g. Street, church)
          ?x wdt:P31  ?class . 
          
          # x is a Street
          #?x wdt:P31 wd:Q79007 .

          # hat Staat Deutschland
          # ?x wdt:P17  ?Q183 .
          


In [136]:

# Query for the map
mapQuery = '''
<iframe style="width: 80vw; height: 50vh; border: none;" 
src="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%20%20%20%20%20%20%20%20
prefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20%20%20%20%20
prefix%20geo%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.opengis.net%2Font%2Fgeosparql%23%3E%0A%20%20%20%20%20%20%20%20
prefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20%20%20%20%20
prefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20%20%20%20%20
prefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20%20%20%20%20
SELECT%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FclassLabel%20%3FotherLocation%20%3Fdist%0A%20%20%20%20%20%20%20%20
WHERE%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20
BIND('Point({longitude}%20{latitude})'%5E%5Egeo%3AwktLiteral%20AS%20%3FcurrentLocation).%0A%20%20%20%20%20%20%20%20%20%20
SERVICE%20wikibase%3Aaround%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP625%20%3FotherLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20
bd%3AserviceParam%20wikibase%3Acenter%20%3FcurrentLocation.%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20
bd%3AserviceParam%20wikibase%3Aradius%20'{radius}'.%20%0A%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%23%20
x%20is%20named%20after%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP138%20%3Fsomebody%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20
which%20Class%20is%20x%20(e.g.%20Street%2C%20church)%0A%20%20%20%20%20%20%20%20%20%20%3Fx%20wdt%3AP31%20%20%3Fclass%20.%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20
x%20is%20a%20Street%0A%20%20%20%20%20%20%20%20%20%20%23%3Fx%20wdt%3AP31%20wd%3AQ79007%20.%0A%0A%20%20%20%20%20%20%20%20%20%20%23%20
hat%20Staat%20Deutschland%0A%20%20%20%20%20%20%20%20%20%20%23%20%3Fx%20wdt%3AP17%20%20%3FQ183%20.%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20Retrieve%20Labels%0A%20%20%20%20%20%20%20%20%20%20
SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22de%22.%20%7D%20%0A%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%23%20
Compute%20Distance%0A%20%20%20%20%20%20%20%20%20%20%23%20
BIND(geof%3Adistance(%3FotherLocation%2C%20%3FcurrentLocation)%20as%20%3Fdist)%20%0A%20%20%20%20%20%20%20%20%7D%20%0A%0A%20%20%20%20%20%20%20%20
LIMIT%20{limit}%0A%20%20%20%20%20%20%20%20%23ORDER%20BY%20%3Fdist" 
referrerpolicy="origin" sandbox="allow-scripts allow-same-origin allow-popups"></iframe>'''.format(longitude = longitude, latitude = latitude, radius = radius, limit = limit)

mapQuery

'\n<iframe style="width: 80vw; height: 50vh; border: none;" \nsrc="https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%20%20%20%20%20%20%20%20\nprefix%20bd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.bigdata.com%2Frdf%23%3E%0A%20%20%20%20%20%20%20%20\nprefix%20geo%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.opengis.net%2Font%2Fgeosparql%23%3E%0A%20%20%20%20%20%20%20%20\nprefix%20wd%3A%20%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fentity%2F%3E%0A%20%20%20%20%20%20%20%20\nprefix%20wdt%3A%20%20%20%20%20%20%3Chttp%3A%2F%2Fwww.wikidata.org%2Fprop%2Fdirect%2F%3E%0A%20%20%20%20%20%20%20%20\nprefix%20wikibase%3A%20%3Chttp%3A%2F%2Fwikiba.se%2Fontology%23%3E%0A%0A%0A%20%20%20%20%20%20%20%20\nSELECT%20%3FxLabel%20%3Fx%20%3FsomebodyLabel%20%3Fsomebody%20%3FclassLabel%20%3FotherLocation%20%3Fdist%0A%20%20%20%20%20%20%20%20\nWHERE%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20\nBIND(\'Point(8.466111111%2049.487777777)\'%5E%5Egeo%3AwktLiteral%20AS%20%3FcurrentLocation).%

In [135]:
# Initializing SPARQL Wrapper and querying
sparql = SPARQLWrapper(endpoint)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


#results
df = pd.DataFrame(results['results']['bindings'])
for col in df.columns:
    df[col] = df[col].apply(lambda x: x['value'])
df.head(5)


,x,otherLocation,somebody,xLabel,somebodyLabel,classLabel
0,http://www.wikidata.org/entity/Q1614987,Point(8.38837 49.3826),http://www.wikidata.org/entity/Q408284,Herz-Jesu-Kirche (Schifferstadt),Heiligstes Herz Jesu,Kirchengebäude
1,http://www.wikidata.org/entity/Q1876995,Point(8.26511 49.482),http://www.wikidata.org/entity/Q128538,Lukaskirche (Birkenheide),Lukas der Evangelist,Kirchengebäude
2,http://www.wikidata.org/entity/Q2323109,Point(8.52368 49.3679),http://www.wikidata.org/entity/Q183332,St. Sebastian (Ketsch),Sebastian,Kirchengebäude
3,http://www.wikidata.org/entity/Q18289282,Point(8.377311 49.37734),http://www.wikidata.org/entity/Q17590,St. Laurentius,Laurentius von Rom,Pfarrkirche
4,http://www.wikidata.org/entity/Q62068843,Point(8.48364 49.3887),http://www.wikidata.org/entity/Q1779411,Kollerinsel,Kollerinsel,Landschaftsschutzgebiet
